In [2]:
#!pip install autogluon
#!pip install pandas

In [3]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random
from sklearn.model_selection import train_test_split

C:\Users\holwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [5]:
x_train_b = pd.read_csv('cleaned_data_quarterly/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data_quarterly/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data_quarterly/B/x_test_b.csv')

x_train_b

,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,calc_day,calc_hour,hour_sin,hour_cos,month_sin,month_cos,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3
0,2019-01-01 00:00:00,5.6,5.6,5.5,5.4,1.239,1.239,1.240,1.241,1226.4,...,NaN,NaN,0.000000,1.000000,0.0,1.000000e+00,-0.0000,-0.0,-0.0,-0.0
1,2019-01-01 01:00:00,5.4,5.4,5.4,5.5,1.241,1.240,1.239,1.239,1157.6,...,NaN,NaN,0.258819,0.965926,0.0,1.000000e+00,-0.0000,-0.0,-0.0,-0.0
2,2019-01-01 02:00:00,5.5,5.4,5.4,5.3,1.238,1.238,1.239,1.239,1087.6,...,NaN,NaN,0.500000,0.866025,0.0,1.000000e+00,-0.0000,-0.0,-0.0,-0.0
3,2019-01-01 03:00:00,5.3,5.3,5.4,5.4,1.240,1.240,1.240,1.239,1016.4,...,NaN,NaN,0.707107,0.707107,0.0,1.000000e+00,-0.0000,-0.0,-0.0,-0.0
4,2019-01-01 04:00:00,5.5,5.6,5.7,5.9,1.239,1.238,1.237,1.236,1029.0,...,NaN,NaN,0.866025,0.500000,0.0,1.000000e+00,-0.0000,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25870,2023-04-30 19:00:00,4.6,4.6,4.5,4.5,1.275,1.276,1.277,1.278,1636.1,...,29.0,7.0,-0.965926,0.258819,1.0,6.123234e-17,0.7652,0.0,-0.0,-0.0
25871,2023-04-30 20:00:00,4.5,4.5,4.5,4.5,1.278,1.279,1.279,1.279,1747.8,...,29.0,7.0,-0.866025,0.500000,1.0,6.123234e-17,-0.0000,-0.0,-0.0,-0.0
25872,2023-04-30 21:00:00,4.5,4.5,4.5,4.5,1.279,1.279,1.279,1.279,1797.7,...,29.0,7.0,-0.707107,0.707107,1.0,6.123234e-17,-0.0000,-0.0,-0.0,-0.0
25873,2023-04-30 22:00:00,4.5,4.5,4.5,4.5,1.279,1.280,1.280,1.280,1534.3,...,29.0,7.0,-0.500000,0.866025,1.0,6.123234e-17,-0.0000,-0.0,-0.0,-0.0


In [7]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)

train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','snow_density:kgm3_Q0','snow_density:kgm3_Q1','snow_density:kgm3_Q2','snow_density:kgm3_Q3','date_forecast'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','location'], axis = 1)

absolute_humidity_2m:gm3_Q0             0
absolute_humidity_2m:gm3_Q1             0
absolute_humidity_2m:gm3_Q2             0
absolute_humidity_2m:gm3_Q3             0
air_density_2m:kgm3_Q0                  0
air_density_2m:kgm3_Q1                  0
air_density_2m:kgm3_Q2                  0
air_density_2m:kgm3_Q3                  0
ceiling_height_agl:m_Q0              5114
ceiling_height_agl:m_Q1              4136
ceiling_height_agl:m_Q2              4140
ceiling_height_agl:m_Q3              4143
clear_sky_energy_1h:J_Q0                0
clear_sky_energy_1h:J_Q1                0
clear_sky_energy_1h:J_Q2                0
clear_sky_energy_1h:J_Q3                0
clear_sky_rad:W_Q0                      0
clear_sky_rad:W_Q1                      0
clear_sky_rad:W_Q2                      0
clear_sky_rad:W_Q3                      0
cloud_base_agl:m_Q0                  2303
cloud_base_agl:m_Q1                  1853
cloud_base_agl:m_Q2                  1857
cloud_base_agl:m_Q3               

# Trying with validation set equal to half of the estimated data


In [ ]:

train_data_b.drop(columns= ['index'], inplace= True)

In [ ]:
observed_zero = train_data_b[train_data_b['observed'] == 0]
observed_zero['forecast_month'].unique()


array([10, 11, 12,  1,  2,  3,  4], dtype=int64)

In [ ]:
from sklearn.model_selection import train_test_split

def split_dataset(train_data, date_column='forecast_month', observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set includes approximately 50% of the estimated data, evenly distributed across months.
    Additionally, it includes about half of the observed data for May, June, and July, if present.
    The training set includes all months, excluding the observed data that is included in the validation set.
    
    :param train_data: The original training dataset as a pandas DataFrame.
    :param date_column: The name of the column that contains the month information.
    :param observed_column: The name of the column that indicates if the data is observed.
    :return: A tuple (training_set, validation_set)
    """
    # Work with a copy to avoid modifying the original DataFrame
    train_data = train_data.copy()
    train_data.sort_values(by='date_forecast', inplace=True)
    
    # Separate observed and estimated data
    estimated_data = train_data[train_data[observed_column] == 0]
    observed_data = train_data[train_data[observed_column] != 0]
    
    # Split the estimated data into training and validation sets
    estimated_train, estimated_val = train_test_split(
        estimated_data, test_size=0.5, random_state=42, stratify=estimated_data[date_column]
    )
    
    # Check if there are any observed data for May, June, and July
    if not observed_data[observed_data[date_column].isin([5, 6, 7])].empty:
        print("hellooo")
        observed_may_june_july = observed_data[observed_data[date_column].isin([5, 6, 7])]
        observed_train_mjj, observed_val_mjj = train_test_split(
            observed_may_june_july, test_size=0.5, random_state=42, stratify=observed_may_june_july[date_column]
        )
    else:
        
        observed_train_mjj = pd.DataFrame()
        observed_val_mjj = pd.DataFrame()
    
    # Combine the estimated and observed May, June, July data for the validation set
    validation_set = pd.concat([estimated_val, observed_val_mjj])
    validation_set.sort_values(by='date_forecast', inplace=True)
    
    # The rest of the observed data (excluding May, June, July) will be added to the training set
    observed_rest = observed_data[~observed_data[date_column].isin([5, 6, 7])]
    
    # Combine all training parts for the final training set
    training_set = pd.concat([estimated_train, observed_train_mjj, observed_rest])
    training_set.sort_values(by='date_forecast', inplace=True)
    
    return training_set, validation_set

# Create a copy of the DataFrame to pass to the function
# train_data_copy = train_data_b.copy()
# train_data_b, val_data_b = split_dataset(train_data_copy, 'forecast_month', 'observed')

# print(len(train_data_b))
# print(len(val_data_b))



hellooo
22174
4768


In [ ]:
# train_data_b.drop(columns=['date_forecast'],inplace=True)
# val_data_b.drop(columns=['date_forecast'],inplace=True)

In [ ]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231107_144658\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231107_144658\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   692.99 GB / 1022.87 GB (67.7%)
Train Data Rows:    22174
Train Data Columns: 189
Tuning Data Rows:    4768
Tuning Data Columns: 189
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1120.3875, -0.0, 86.31003, 188.40661)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predi

[1000]	valid_set's l1: 18.3634
[2000]	valid_set's l1: 17.7256
[3000]	valid_set's l1: 17.5069
[4000]	valid_set's l1: 17.3894
[5000]	valid_set's l1: 17.3616
[6000]	valid_set's l1: 17.3326
[7000]	valid_set's l1: 17.3073
[8000]	valid_set's l1: 17.2897
[9000]	valid_set's l1: 17.2769
[10000]	valid_set's l1: 17.265
[1000]	valid_set's l1: 22.1418
[2000]	valid_set's l1: 21.4247
[3000]	valid_set's l1: 21.1783
[4000]	valid_set's l1: 21.0363
[5000]	valid_set's l1: 20.9557
[6000]	valid_set's l1: 20.8921
[7000]	valid_set's l1: 20.8589
[8000]	valid_set's l1: 20.8437
[9000]	valid_set's l1: 20.8245
[10000]	valid_set's l1: 20.8043
[1000]	valid_set's l1: 19.7428
[2000]	valid_set's l1: 19.2393
[3000]	valid_set's l1: 19.0696
[4000]	valid_set's l1: 19.0004
[5000]	valid_set's l1: 18.9363
[6000]	valid_set's l1: 18.8906
[7000]	valid_set's l1: 18.8722
[8000]	valid_set's l1: 18.8427
[9000]	valid_set's l1: 18.8268
[10000]	valid_set's l1: 18.8149
[1000]	valid_set's l1: 18.9323
[2000]	valid_set's l1: 18.394
[3000]	

	-39.4857	 = Validation score   (-mean_absolute_error)
	261.03s	 = Training   runtime
	4.9s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 708.08s of the 1207.75s of remaining time.
	Memory not enough to fit LGBModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 18.4614
[2000]	valid_set's l1: 18.3364
[1000]	valid_set's l1: 22.7783
[2000]	valid_set's l1: 22.6277
[3000]	valid_set's l1: 22.5922
[4000]	valid_set's l1: 22.5873
[5000]	valid_set's l1: 22.5763
[6000]	valid_set's l1: 22.5752
[7000]	valid_set's l1: 22.5723
[8000]	valid_set's l1: 22.5707
[9000]	valid_set's l1: 22.5704
[10000]	valid_set's l1: 22.5697
[1000]	valid_set's l1: 20.3171
[2000]	valid_set's l1: 20.1776
[3000]	valid_set's l1: 20.1422
[4000]	valid_set's l1: 20.1453
[1000]	valid_set's l1: 19.7635
[2000]	valid_set's l1: 19.6227
[3000]	valid_set's l1: 19.5684
[1000]	valid_set's l1: 20.9382
[2000]	valid_set's l1: 20.7355
[3000]	valid_set's l1: 20.7037
[1000]	valid_set's l1: 20.7531
[2000]	valid_set's l1: 20.6766
[3000]	valid_set's l1: 20.6432
[4000]	valid_set's l1: 20.6335
[5000]	valid_set's l1: 20.6289
[6000]	valid_set's l1: 20.6327
[1000]	valid_set's l1: 21.7812
[2000]	valid_set's l1: 21.5911
[3000]	valid_set's l1: 21.5435
[4000]	valid_set's l1: 21.5222
[5000]	

	-40.8652	 = Validation score   (-mean_absolute_error)
	184.17s	 = Training   runtime
	1.92s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 514.93s of the 1014.6s of remaining time.
	-44.6209	 = Validation score   (-mean_absolute_error)
	76.01s	 = Training   runtime
	2.06s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 435.83s of the 935.5s of remaining time.
	Memory not enough to fit CatBoostModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 2310.
	Ran out of time, early stopping on iteration 2584.
	Ran out of time, early stopping on iteration 2773.
	Ran out of time, early stopping on iteration 2888.
	Ran out of time, early stopping on iteration 

[1000]	valid_set's l1: 17.5645
[1000]	valid_set's l1: 20.4733


	-39.9807	 = Validation score   (-mean_absolute_error)
	19.45s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 482.38s of the 482.33s of remaining time.
	Memory not enough to fit LGBModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-40.0805	 = Validation score   (-mean_absolute_error)
	11.68s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 470.05s of the 470.0s of remaining time.
	-40.103	 = Validation score   (-mean_absolute_error)
	69.08s	 = Training   runtime
	1.62s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 398.71s of the 398.66s of remaining time.
	Memory not enough to fit CatBoostModel folds

In [ ]:
predictor_b.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-39.071064,12.543318,1198.722158,0.000000,0.185030,3,True,17
1,NeuralNetFastAI_BAG_L2,-39.264160,12.366797,1158.965005,0.483112,212.942932,2,True,14
2,WeightedEnsemble_L2,-39.475335,7.664803,445.544492,0.000000,0.159031,2,True,8
3,LightGBMXT_BAG_L1,-39.485701,4.902304,261.029939,4.902304,261.029939,1,True,3
4,LightGBMLarge_BAG_L2,-39.762324,12.060205,985.594195,0.176521,39.572123,2,True,16
5,XGBoost_BAG_L2,-39.806729,12.108225,973.654009,0.224540,27.631937,2,True,15
6,LightGBMXT_BAG_L2,-39.980705,12.072204,965.476062,0.188519,19.453990,2,True,9
7,LightGBM_BAG_L2,-40.080502,11.988696,957.705135,0.105011,11.683063,2,True,10
8,ExtraTreesMSE_BAG_L2,-40.083908,13.835861,963.106674,1.952176,17.084602,2,True,13
9,CatBoost_BAG_L2,-40.102727,11.977684,1036.707535,0.094000,90.685463,2,True,12


In [ ]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3_Q0      0
absolute_humidity_2m:gm3_Q1      0
absolute_humidity_2m:gm3_Q2      0
absolute_humidity_2m:gm3_Q3      0
                                ..
direct_rad_x_sun_elevation_Q3    0
time                             0
index                            0
pv_measurement                   0
observed                         0
Length: 197, dtype: int64


In [ ]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

,pv_measurement
0,0.349458
1,0.457059
2,0.336891
3,2.971397
4,47.737179
...,...
715,42.718563
716,11.937856
717,0.695537
718,0.008004


In [ ]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

,pv_measurement
0,0.349458
1,0.457059
2,0.336891
3,2.971397
4,47.737179
...,...
715,42.718563
716,11.937856
717,0.695537
718,0.008004


In [ ]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_b.to_csv(os.path.join(output_dir, 'y_pred_b.csv'))